In [1]:
!pip -q install sentence-transformers openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.7 MB/s eta 0:00:00


In [2]:
# -*- coding: utf-8 -*-
# Hybrid LLM + SBERT for skills→occupations, taxonomy, relations
import os, re, json, numpy as np, pandas as pd
from typing import List, Any, Optional
from kaggle_secrets import UserSecretsClient
from tqdm import tqdm
from openai import OpenAI
from types import SimpleNamespace
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import KMeans

# ---------------- Core IO ----------------
def get_client():
    user_secrets = UserSecretsClient()
    k = user_secrets.get_secret("OPENAI_API_KEY")  # ไปตั้งค่า secret ใน Kaggle ก่อน
    b = None
    try:
        b = user_secrets.get_secret("OPENAI_BASE_URL")
    except Exception:
        pass
    if not k:
        raise RuntimeError("OPENAI_API_KEY missing in Kaggle Secrets")
    return OpenAI(api_key=k, base_url=b) if b else OpenAI(api_key=k)

def ensure_dir(p: str): os.makedirs(p, exist_ok=True)
def dumpj(p: str, o: Any):
    ensure_dir(os.path.dirname(p))
    with open(p, "w", encoding="utf-8") as f: json.dump(o, f, ensure_ascii=False, indent=2)

def pick_col(df: pd.DataFrame, prefs: List[str]) -> str:
    for c in prefs:
        if c in df.columns: return c
    objs = [c for c in df.columns if df[c].dtype == object]
    return objs[0] if objs else df.columns[0]

def _slug(s: str) -> str:
    s = re.sub(r"[^\w\- ]+", "", s).strip().replace("-", "_")
    s = re.sub(r"\s+", "_", s)
    if not s: s = "unnamed"
    if s[0].isdigit(): s = "_" + s
    return s

# ---------------- Prompts ----------------
SYS_A = (
    "You are an ontology labeler. Given a SKILL term and a closed set of OCCUPATION labels, "
    "assign ALL occupations from the list that could plausibly require the skill. "
    "Be generous. Return strictly a JSON array of strings from the list. Never return an empty array."
)
SYS_C = (
    "You are an ontology relation classifier. Given two OCCUPATION labels and a closed set of non-taxonomic "
    "relations (e.g., collaborates_with, depends_on, related_to), return exactly ONE label."
)
AUTO_SYS_B = (
    "You design occupation taxonomies.\n"
    "Given a list of OCCUPATION labels, create HIGH-LEVEL PARENT categories and map each child to exactly ONE parent.\n"
    "OUTPUT STRICTLY JSON ONLY: an array of [child, parent] pairs. No commentary."
)

# ---------------- LLM helpers ----------------
def chat(client: OpenAI, model: str, sys: str, usr: str, mt: int = 512) -> str:
    r = client.chat.completions.create(
        model=model,
        messages=[{"role": "system", "content": sys}, {"role": "user", "content": usr}],
        temperature=0,
        max_tokens=mt,
    )
    return r.choices[0].message.content.strip()

def _extract_json_array(txt: str):
    m = re.search(r"\[.*?\]", txt, re.S)
    if not m: return None
    try:
        arr = json.loads(m.group(0))
        return arr if isinstance(arr, list) else [arr]
    except Exception:
        return None

def _json_only(text: str):
    m = re.search(r"\{.*\}", text, re.S)
    if not m: return None
    try: return json.loads(m.group(0))
    except Exception: return None

# ---------------- Embedding layer ----------------
EMB_NAME = os.getenv("EMB_MODEL_NAME") or "sentence-transformers/all-MiniLM-L6-v2"
_emb_model = SentenceTransformer(EMB_NAME)

def emb_encode(texts: List[str]): return _emb_model.encode(texts, convert_to_tensor=True)
def emb_cos(a, b): return util.cos_sim(a, b)

# ---------------- Param selection by LLM ----------------
def _score_stats(arr: np.ndarray, topn: int = 30) -> dict:
    a = np.asarray(arr).astype("float32")
    if a.size == 0: a = np.array([0.0], dtype="float32")
    a_sorted = np.sort(a)[::-1]
    top = a_sorted[:topn].tolist()
    return {
        "n": int(a.size),
        "mean": float(a.mean()),
        "std": float(a.std()),
        "min": float(a.min()),
        "max": float(a.max()),
        "top": top,
        "gaps": [float(top[i] - top[i+1]) for i in range(len(top)-1)]
    }

def _llm_pick_k_thr_for_skill(client: OpenAI, model: str, skill: str, scores: np.ndarray, n_occ: int):
    stats = _score_stats(scores, topn=min(30, len(scores)))
    sys = "You choose hyperparameters for candidate selection from similarity scores. Output strictly one JSON."
    usr = (
        "We map a SKILL to a subset of OCCUPATIONS using cosine similarity.\n"
        f"Skill: {skill}\nNum occupations: {n_occ}\n"
        "Similarity summary:\n" + json.dumps(stats, ensure_ascii=False, indent=2) + "\n"
        'Return JSON: {"k": int in [1,n_occ], "threshold": float in [0,1]}.\n'
        "Pick k at elbow/large-gap. threshold is minimal include score. "
        "Prefer small k if gaps are large. Prefer larger k if decay is smooth."
    )
    try:
        obj = _json_only(chat(client, model, sys, usr, 256)) or {}
        k = int(obj.get("k", 0)); thr = float(obj.get("threshold", -1))
        if 1 <= k <= n_occ and 0.0 <= thr <= 1.0: return k, thr
    except Exception:
        pass
    # fallback
    top = stats["top"]; gaps = np.array(stats["gaps"]) if stats["gaps"] else np.array([])
    k_fb = max(1, int(np.argmax(gaps)) + 1) if gaps.size else min(5, n_occ)
    thr_fb = float(max(0.0, min(0.95, np.percentile(scores, 85)))) if scores.size else 0.5
    return k_fb, thr_fb

def _llm_pick_sim_bounds(client: OpenAI, model: str, pair_sims: np.ndarray):
    stats = {
        "n": int(pair_sims.size),
        "mean": float(pair_sims.mean()),
        "std": float(pair_sims.std()),
        "p25": float(np.percentile(pair_sims, 25)),
        "p50": float(np.percentile(pair_sims, 50)),
        "p75": float(np.percentile(pair_sims, 75)),
        "p90": float(np.percentile(pair_sims, 90)),
        "min": float(pair_sims.min()),
        "max": float(pair_sims.max()),
    }
    sys = "You set decision thresholds from a similarity distribution. Output strictly one JSON."
    usr = (
        "We classify relations between OCCUPATION pairs using cosine similarity as soft evidence.\n"
        "Pick two thresholds with 0 ≤ sim_low < sim_high ≤ 1.\n"
        "Above sim_high: auto-label 'related_to'. Below sim_low: defer to model.\n"
        "Similarity summary:\n" + json.dumps(stats, ensure_ascii=False, indent=2) +
        '\nReturn JSON: {"sim_high": float, "sim_low": float}.'
    )
    try:
        obj = _json_only(chat(client, model, sys, usr, 128)) or {}
        hi = float(obj.get("sim_high", -1)); lo = float(obj.get("sim_low", -1))
        if 0 <= lo < hi <= 1: return hi, lo
    except Exception:
        pass
    return float(stats["p90"]), float(stats["p25"])

# ---------------- Task A ----------------
def taskA_map_skills_to_occ(skills_csv: str, occ_csv: str, out_dir: str, model: str) -> List[str]:
    df_s = pd.read_csv(skills_csv)
    df_o = pd.read_csv(occ_csv)
    s_col = pick_col(df_s, ["skill", "skills", "name", "title"])
    o_col = pick_col(df_o, ["occupation", "job", "title", "name"])

    skills = [str(x).strip() for x in df_s[s_col].dropna().tolist() if str(x).strip()]
    occup = list(dict.fromkeys([str(x).strip() for x in df_o[o_col].dropna().tolist() if str(x).strip()]))

    client = get_client()
    occ_emb_t = emb_encode(occup)
    data = []
    dbg_dir = os.path.join(out_dir, "_debug_raw"); ensure_dir(dbg_dir)

    for s in tqdm(skills, desc="TaskA: skills→occupations"):
        s_emb_t = emb_encode([s])
        scores = emb_cos(s_emb_t, occ_emb_t)[0].cpu().numpy()

        k_candidate, sim_threshold = _llm_pick_k_thr_for_skill(client, model, s, scores, len(occup))

        top_idx = np.argsort(-scores)[:k_candidate].tolist()
        cand_labels = [occup[i] for i in top_idx]
        thr_keep = [occup[i] for i in range(len(occup)) if scores[i] >= sim_threshold]
        cand_labels = list(dict.fromkeys(cand_labels + thr_keep))

        tmpl = (
            "Skill: {skill}\n"
            "Candidate occupation labels (from similarity-based selection): {labels}\n"
            "Answer with ALL matching occupations strictly as a JSON array of strings from the candidate list."
        )
        u = tmpl.format(skill=s, labels=", ".join(cand_labels))
        txt = ""
        try:
            txt = chat(client, model, SYS_A, u, 512)
            labs = _extract_json_array(txt) or []
        except Exception:
            labs = []

        labs = [lab for lab in labs if lab in cand_labels]
        if not labs: labs = thr_keep or cand_labels[: min(3, len(cand_labels))]

        try:
            with open(os.path.join(dbg_dir, f"{_slug(s)[:80]}.txt"), "w", encoding="utf-8") as f:
                f.write(txt if isinstance(txt, str) else str(txt))
        except Exception:
            pass

        for lab in labs: data.append({"text": s, "label": lab})

    dumpj(f"{out_dir}/data.json", data)
    dumpj(f"{out_dir}/label_mapper.json", {str(i): l for i, l in enumerate(occup)})
    dumpj(
        f"{out_dir}/templates.json",
        ["Classify the SKILL into one or more OCCUPATION labels. Term: {text}. Labels: {labels}. Answer with a JSON array of labels."]
    )

    occ2skills = {occ: [] for occ in occup}
    for r in data: occ2skills[r["label"]].append(r["text"])
    occ2skills = {occ: sorted(dict.fromkeys(sk_list)) for occ, sk_list in occ2skills.items()}
    dumpj(f"{out_dir}/occ2skills.json", [{"occupation": occ, "skills": sks} for occ, sks in occ2skills.items()])

    ents = [{"entity": sk, "type": occ} for occ, sks in occ2skills.items() for sk in sks]
    per = {occ: len(occ2skills[occ]) for occ in occup}
    dumpj(f"{os.path.dirname(out_dir)}/jobskillsset_entities.json", ents)
    dumpj(f"{os.path.dirname(out_dir)}/stats.json", {"kb_name": "JobSkillsSet", "n_entities": len(ents), "n_types": len(occup), "per_type": per})
    return occup

# ---------------- Task B (embedding-aided seeds + LLM) ----------------
def _cluster_and_name_with_llm(client: OpenAI, model: str, occ_types: List[str]) -> List[List[str]]:
    n = len(occ_types)
    k = min(8, max(2, int(np.sqrt(n)))) if n > 8 else max(2, min(4, n))
    X = emb_encode(occ_types).cpu().numpy()
    km = KMeans(n_clusters=k, n_init="auto", random_state=42)
    cl = km.fit_predict(X)
    reps, parent_names = [], []
    for c in range(k):
        idx = np.where(cl == c)[0].tolist()
        sample = [occ_types[i] for i in idx[:8]]
        reps.append(sample)
    for sample in reps:
        name = chat(get_client(), model, "Name taxonomic parent",
                    "Name a high-level OCCUPATION parent for:\n- " + "\n- ".join(sample) + "\nReturn only the name.", 32)
        name = re.sub(r"[\n\[\]\{\}\"]", "", name).strip() or "Category"
        parent_names.append(name)
    return [[occ_types[i], parent_names[cl[i]]] for i in range(n)]

def llm_propose_taxonomy_seeds(client: OpenAI, model: str, occ_types: List[str]) -> List[List[str]]:
    usr = "Occupations:\n" + "\n".join(f"- {o}" for o in occ_types)
    for _ in range(2):
        txt = chat(client, model, AUTO_SYS_B, usr, mt=1024)
        arr = _extract_json_array(txt)
        ok = []
        if isinstance(arr, list):
            for p in arr:
                if isinstance(p, list) and len(p) == 2 and all(isinstance(x, str) and x.strip() for x in p):
                    ok.append([p[0].strip(), p[1].strip()])
        if ok: return ok
    try: return _cluster_and_name_with_llm(client, model, occ_types)
    except Exception: pass
    # simple fallback
    buckets = {
        "Software & Engineering": ("Engineer","Developer","Programmer","Architect","Software"),
        "Analytics": ("Analyst","Science","Scientist","BI","Intelligence"),
        "IT Operations": ("Administrator","Network","Systems"),
        "Management": ("Manager","Project","Program","Product"),
    }
    pairs = []
    for o in occ_types:
        parent = "Other"
        low = o.lower()
        for b,kws in buckets.items():
            if any(k.lower() in low for k in kws): parent=b; break
        pairs.append([o,parent])
    return pairs

def taskB_occ_taxonomy(occ_types: List[str], out_dir: str, model: str, seeds: Optional[List[List[str]]] = None):
    pairs = [{"child": c, "parent": p} for c,p in (seeds or [(c,"Occupation") for c in occ_types])]
    dumpj(f"{out_dir}/pairs.json", pairs)
    dumpj(f"{out_dir}/label_mapper.json", {"0": "is-a"})
    dumpj(f"{out_dir}/template.json",
          ["Decide if child is-a parent. Answer only 'yes' or 'no'.\nChild: {child}\nParent: {parent}\nAnswer:"])

# ---------------- Task C (relations with LLM-calibrated bounds) ----------------
def taskC_occ_relations(occ_types: List[str], out_dir: str, model: str, rel_labels: List[str], max_pairs: int = 120):
    client = get_client()
    pairs = [(occ_types[i], occ_types[j]) for i in range(len(occ_types)) for j in range(i+1, len(occ_types))]
    pairs = pairs[:max_pairs]

    # estimate similarity distribution from a sample
    occ_emb_np = emb_encode(occ_types).cpu().numpy()
    rng = np.random.default_rng(42)
    if len(occ_types) > 1:
        idxs = rng.choice(len(occ_types), size=min(200, len(occ_types)), replace=False)
        sims = []
        for i in range(len(idxs)):
            for j in range(i+1, len(idxs)):
                a, b = occ_emb_np[idxs[i]], occ_emb_np[idxs[j]]
                s = float(np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b)+1e-9))
                sims.append(s)
        sims = np.array(sims, dtype="float32") if sims else np.array([0.5], dtype="float32")
    else:
        sims = np.array([0.5], dtype="float32")

    sim_high, sim_low = _llm_pick_sim_bounds(client, model, sims)

    tmpl = (
        "Head OCCUPATION: {h}\nTail OCCUPATION: {t}\n"
        "Candidate relation labels: {labels}\n"
        "Embedding cosine similarity between head and tail: {sim:.3f}\n"
        "Use the similarity as soft evidence only. Return exactly ONE label."
    )

    name2idx = {n:i for i,n in enumerate(occ_types)}
    occ_emb_t = emb_encode(occ_types)

    out = []
    for h, t in tqdm(pairs, desc="TaskC: relations"):
        sim = float(emb_cos(occ_emb_t[name2idx[h]], occ_emb_t[name2idx[t]]).cpu().numpy()[0][0])
        if sim >= sim_high:
            lab = "related_to"
        elif sim <= sim_low:
            lab = chat(client, model, SYS_C, tmpl.format(h=h,t=t,labels=", ".join(rel_labels), sim=sim), 8)
            if lab not in rel_labels: lab = rel_labels[0]
        else:
            lab = chat(client, model, SYS_C, tmpl.format(h=h,t=t,labels=", ".join(rel_labels), sim=sim), 8)
            if lab not in rel_labels: lab = "related_to"
        out.append({"head": h, "tail": t, "label": lab, "sim": round(sim,3)})

    dumpj(f"{out_dir}/pairs.json", out)
    dumpj(f"{out_dir}/label_mapper.json", {str(i): l for i,l in enumerate(rel_labels)})
    dumpj(f"{out_dir}/templates.json",
          ["Classify the relation between Head and Tail strictly as one of: {labels}.\nHead: {head}\nTail: {tail}\nAnswer:"])

# ---------------- CLI ----------------
def main():
    args_dict = {
        "occupations_csv": "/kaggle/input/ontology/occ_update.csv",
        "skills_csv": "/kaggle/input/ontology/skill_update.csv",
        "out_root": "Occupations_Skills_Mapping_Sbert_LLM4o",
        "model": os.getenv("OPENAI_CHAT_MODEL") or "gpt-4o",
        "taxonomy_seeds": 'auto',
        "taxonomy_mode": "auto",
        "relation_labels": ["related_to", "collaborates_with", "depends_on"],
    }
    args = SimpleNamespace(**args_dict)

    # Task A
    A_dir = f"{args.out_root}/TaskA/JobSkillsSet"
    occ_types = taskA_map_skills_to_occ(args.skills_csv, args.occupations_csv, A_dir, args.model)

    # Task B seeds
    auto = args.taxonomy_mode == "auto" or (
        isinstance(args.taxonomy_seeds, str) and args.taxonomy_seeds.strip().lower() == "auto"
    )
    if auto:
        client = get_client()
        seeds = llm_propose_taxonomy_seeds(client, args.model, occ_types)
    else:
        try: seeds = json.loads(args.taxonomy_seeds)
        except Exception: seeds = []

    dbg_dir = f"{args.out_root}/TaskB/_debug"; ensure_dir(dbg_dir)
    dumpj(f"{dbg_dir}/auto_seeds.json", seeds or [])

    # Task B
    B_dir = f"{args.out_root}/TaskB/Occupations"
    taskB_occ_taxonomy(occ_types, B_dir, args.model, seeds)

    # Task C
    C_dir = f"{args.out_root}/TaskC/Occupations"
    taskC_occ_relations(occ_types, C_dir, args.model, args.relation_labels)

    print(f"Done → {args.out_root}")

if __name__ == "__main__":
    main()


2025-09-24 10:12:05.615616: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758708725.945026      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758708726.038003      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

TaskC: relations: 100%|██████████| 45/45 [00:19<00:00,  2.25it/s]

Done → Occupations_Skills_Mapping_Sbert_LLM4o
